In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [6]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.5821372185050036, Any[sparse([5], [1], [1.5259583460959916], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [9]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> W .* 2, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])


size(W.val) = (2, 1)
(2, 1)

[2.0; 2.0;;]

In [19]:
W = rand(2,1)
@show size(((W' * W) .* W)' * W)

U1, C1 = ⬅Dual(adjoint, W)
@show size(U1)
U2, C2 = ⬅Dual(*, U1, W)
@show size(U2)
U3, C3 = ⬅Dual(Base.broadcasted, *, U2, W)
@show size(U3)
U4, C4 = ⬅Dual(adjoint, U3)
@show size(U4)
U5, C5 = ⬅Dual(*, U4, W)
@show size(U5)

@show DU4, _ = C5([1])
@show DU3, _ = C4(DU4)



"""
ctx = Utero.⬅Context()
W1, W2, W3, W4 = AddParams!(ctx, rand(2,1), rand(2,1), rand(2,1), rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> ((W' ) .* W3)' * W4, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])
"""

size(((W' * W) .* W)' * W) = (1, 1)
size(U1) = (1, 2)
size(U2) = (1, 1)
size(U3) = (2, 1)
size(U4) = (1, 2)
size(U5) = (1, 1)
(DU4, _) = C5([1]) = ([0.45575614090997796 0.18589359916988146], [0.11041608136267347, 0.04503645903653576])


"ctx = Utero.⬅Context()\nW1, W2, W3, W4 = AddParams!(ctx, rand(2,1), rand(2,1), rand(2,1), rand(2,1))\n(out, grads) = ForwardBackward!(ctx, _ -> ((W' ) .* W3)' * W4, nothing)\n@show size(W.val)\nprint(size(grads[1]))\nprint(grads[1])\n"

In [11]:
ctx = Utero.⬅Context()
W1, W2, W3 = AddParams!(ctx, rand(30,30), rand(40,30), rand(30,40))

function model(X)
    return  ReLU(W3 * ReLU(W2 * ReLU(W1 * X)))
end

X = rand(1500, 30)
Y = mapslices(row -> cos.(row), X, dims=2) 

println(ctx.Params[1].val)
for (x, y) in zip(eachrow(X), eachrow(Y))
    println(MeanSquaredError(model(x), y).val)
    (out, grads) = ForwardBackward!(ctx, x -> MeanSquaredError(model(x), y), x)
    GradientStep!(0.0001, ctx.Params, grads)
    exit("")
end
println(ctx.Params[1].val)


[0.34651270826681835 0.9856295177478517 0.8036310281073437 0.22311991191082725 0.7278612459743528 0.5901010470150359 0.9357265083165979 0.3437512852988758 0.547767763813984 0.6930178889225227 0.22898113150636357 0.3777563621645109 0.5756785857151082 0.014867957554544398 0.51654451021533 0.2503850728318121 0.40821900384264376 0.43364331065609685 0.5504663720616061 0.08516397987370083 0.361644309586992 0.7783277576240416 0.1967889925091122 0.5415244840083794 0.2791213791213575 0.06519504300543155 0.10364972105972314 0.9508683626834388 0.8887139354429184 0.433891474981996; 0.008625475144588113 0.7911120403228118 0.24149661176187875 0.9196481078580113 0.44604123798560213 0.785318559218551 0.44799738556787283 0.2871721467262559 0.37002589665342744 0.6317848949640811 0.20471938513624954 0.21322393310195686 0.906649839018376 0.6596777410653579 0.9950514677181218 0.5592666286962836 0.670763755176304 0.1519405946512984 0.7763812362009457 0.7960319593087412 0.48234155097712383 0.4632412818778002

 0.4507672319356315 0.6773258344278937 0.34127380472972146 0.31360190089657747 0.7003389395587094 0.9545030448023236 0.8479503069906283 0.483827782709209; 0.8417793421182252 0.6726362824821464 0.3297706365561265 0.0019098247298331428 0.960310370764816 0.21399588507795997 0.5003614663037117 0.966086046585877 0.9629605989937892 0.5796843156000416 0.08313128501694966 0.07497322687088526 0.7024742902616253 0.8476771734316135 0.9302390243999885 0.05059386061273086 0.5056210800226046 0.47440489519489404 0.9130042142735547 0.01537268894088939 0.5010913991084253 0.5153834770605341 0.30348380482165804 0.7841825233733541 0.08562251126690645 0.3183302745555687 0.7332824920043616 0.6565801453957943 0.7568971106387593 0.15405854751977066; 0.845813573245658 0.9593371412015635 0.6042602682869878 0.1109566866492977 0.8826099918454302 0.0821177992402563 0.5801770098632338 0.15286035996569036 0.9665885798912139 0.4301816827613927 0.45304892541280306 0.5459411583398325 0.7790079499466656 0.93745381604366

1.4429636519555953e8


MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int32

Closest candidates are:
  convert(::Type{T}, !Matched::Ptr) where T<:Integer
   @ Base pointer.jl:23
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


In [12]:
using MLDatasets

TrainData = MNIST(:train)
ValidData = MNIST(:test)

ctx = ⬅Context()

densenet = Sequential(Dense(28*28, 16, ReLU), Dense(16, 16, ReLU), Dense(16, 10, ReLU), Softmax)
#display(densenet(rand(28*28, 1)))
#display(MeanSquaredError(densenet(rand(28*28, 1)), rand(10, 1)))
AddParams!(ctx, densenet)
println(typeof(densenet.Sequence[1].W))
for (x, y) in DataIterator(TrainData.features, TrainData.targets)
    x = reshape(x, 28*28, 1)
    y = OneHot(Int64(y+1), 10)
    y = reshape(y, 10, 1)
    display(densenet.Sequence[1].W)
    (out, paramgrads) = ForwardBackward!(ctx, x -> MeanSquaredError(densenet(x), y), x)
    GradientStep!(0.0001, ctx.Params, paramgrads)
    display(paramgrads[1])
    display(densenet.Sequence[1].W)
    exit("")
end

Main.Utero.⬅Tracker{Matrix{Float64}}([-0.3469630751484771 0.5313081447819742 … -0.9562413312599197 1.825677303109236; 0.8418087445750742 2.214035053798167 … 0.9808913924479944 -1.039309184476938; … ; 1.5454527132324807 0.13099105485446538 … 0.5034542882117231 2.549908590585409; 2.025037043484334 0.2567228824536351 … -0.2730202410277949 0.7406772154333034], UUID("33023f88-c86d-11ee-25be-034fe8803ba3"), Any[], Any[])

Main.Utero.⬅Tracker{Matrix{Float64}}


UndefVarError: UndefVarError: `exp` not defined